<a href="https://colab.research.google.com/github/ace-racer/Extending-Board-Games-using-deep-learning/blob/master/chess_piece_detection/Siamese_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
"""
Initial code from: https://sorenbouma.github.io/blog/oneshot/

"""
import keras
from keras.layers import Input, Conv2D, Lambda, average, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

input_shape = (100, 100, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)

#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(32,(5,5),activation='relu',input_shape=input_shape,
                   kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(64,(4,4),activation='relu',
                   kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(1024,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))

#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

#merge two encoded inputs with the average
both = average([encoded_l,encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(both)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
#optimizer = SGD(0.0004,momentum=0.6,nesterov=True,decay=0.0003)

optimizer = Adam(0.00006)

siamese_net.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

print(siamese_net.count_params())
print(siamese_net.summary())

47485505
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 1024)         47484480    input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
average_2 (Average)             (None, 1024)         0           sequential_2[1][0]               
 

In [2]:
! git clone https://github.com/ace-racer/Chess-Pieces-Data.git
! ls

fatal: destination path 'Chess-Pieces-Data' already exists and is not an empty directory.
Chess-Pieces-Data  logs  sample_data  weights


In [0]:
import numpy as np
import keras
import os
import itertools
import random
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# change as required
IMAGES_LOCATION = "Chess-Pieces-Data/crawled_1901/"

samples_per_type = {"b": 30, "n": 25, "k": 25, "p": 35, "q": 25, "r": 35}
#samples_per_type = {"b": 3, "n": 2, "k": 2, "p": 3, "q": 2, "r": 3}

# training parameters
IMAGE_SIZE = (100, 100)
CHECKPOINTS_LOCATION = "weights"
LOGS_LOCATION = "logs"
BATCH_SIZE = 32
NUM_EPOCHS = 50

if not os.path.exists(CHECKPOINTS_LOCATION):
    os.makedirs(CHECKPOINTS_LOCATION)

if not os.path.exists(LOGS_LOCATION):
    os.makedirs(LOGS_LOCATION)

X_train_original = []
y_train_original = []


training_images = os.path.join(IMAGES_LOCATION, "train")
validation_images = os.path.join(IMAGES_LOCATION, "test")


files_with_labels = []

for type_name in samples_per_type:
    piece_type_folder = os.path.join(training_images, type_name)
    for idx, f in enumerate(os.listdir(piece_type_folder)):
        if idx >= samples_per_type[type_name]:
            break

        img_file_loc = os.path.join(piece_type_folder, f)
        files_with_labels.append((img_file_loc, type_name))


random.shuffle(files_with_labels)
# print(files_with_labels)

cartesian_product = itertools.product(files_with_labels, files_with_labels)
# print(cartesian_product)

for item1, item2 in cartesian_product:

    img1 = cv2.imread(item1[0])
    img1 = cv2.resize(img1, IMAGE_SIZE, interpolation=cv2.INTER_AREA)

    img2 = cv2.imread(item2[0])
    img2 = cv2.resize(img2, IMAGE_SIZE, interpolation=cv2.INTER_AREA)

    label = int(item1[1] == item2[1])
    X_train_original.append(np.array([img1, img2]))
    y_train_original.append(label)

X_train_original = np.array(X_train_original)
plt.imshow(X_train_original[0][0])
plt.imshow(X_train_original[0][1])

X_train_original = X_train_original.astype('float32')
X_train_original /= 255

plt.imshow(X_train_original[0][0])
plt.imshow(X_train_original[0][1])

y_train_original = np.array(y_train_original)

print(X_train_original.shape)
print(y_train_original.shape)

# split into train and validation splits
X_train, X_test, y_train, y_test = train_test_split(X_train_original, y_train_original, test_size=0.25, random_state=42, stratify = y_train_original)

#X_train = np.array(X_train)
#X_test = np.array(X_test)
#y_train = np.array(y_train)
#y_test = np.array(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train_left = X_train[:, 0, ...]
X_train_right = X_train[:, 1, ...]
print(X_train_left.shape)
print(X_train_right.shape)

X_test_left = X_test[:, 0, ...]
X_test_right = X_test[:, 1, ...]

filepath = os.path.join(CHECKPOINTS_LOCATION, "siamese.hdf5")

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

earlystop = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=10, verbose=1, mode='max')

tensorboard = TensorBoard(log_dir=LOGS_LOCATION, histogram_freq=0, write_graph=True, write_images=True)

callbacks_list = [checkpoint, earlystop, tensorboard]

X_train_instances = [X_train_left, X_train_right]
# X_test_instances = [X_test_left, X_test_right]
hist = siamese_net.fit(X_train_instances, y_train, shuffle=True, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1, validation_split = 0.25, callbacks=callbacks_list)

In [0]:
!ls

In [0]:
from google.colab import files

!zip -r weights.zip weights
!zip -r logs.zip logs



In [0]:
!ls

In [0]:
files.download("weights.zip")

In [0]:
files.download("logs.zip")